# Project 2. MSI.
## Alejandro Varela Rial.

This project aims to produce a file which, once superposed with the simulated protein in VMD, shows the water potential of the system. This is achieved by creating an imaginary 3D grid around the simulations and counting how many times oxigen atoms from water molecules appear in a voxel of this 3D grid along all the frames in the simulations. After normalize and use the chemical potential ecuation one can obtain a .cube file with writeVoxels function. 

In [1]:
from htmd import *
from htmd.molecule.util import maxDistance,writeVoxels

New HTMD version (1.0.23) is available. You are currently on (1.0.16). Use 'conda update htmd' to update to the new version.


In [2]:
def dense(molecule,dmatrix=None):
    '''Calculates water distribution of a simulation and returns it as a 3D matrix.
    If a matrix is given, water distribution of given molecule object is summed to this matrix'''
    dist=round(maxDistance(molecule)) #Maximum distance from origin gives us an idea about how big the matrix will need to be.
    trajwat=molecule.copy()
    trajwat.filter('water and name OH2') #pick only oxygen atoms from the simulation.
    if dmatrix is not None:
        densitymat=dmatrix
    else:
        densitymat = np.zeros((dist+10,dist+10,dist+10))
    #main loop, iterates through atoms and frames in a given simulation.
    for atom in range(0,trajwat.coords.shape[0]): #atoms
        for frame in range(0,trajwat.coords.shape[2]): #frames
            try:
                x=int(trajwat.coords[atom][0][frame]) #integers allows the division of the 3D imaginary grid in 1*1*1 voxels.
                y=int(trajwat.coords[atom][1][frame])
                z=int(trajwat.coords[atom][2][frame])
                if x<0 or y<0 or z<0:
                    continue #avoid negative indexing in the matrix.
                densitymat[x][y][z]=densitymat[x][y][z]+1

            except IndexError:
                continue

    return densitymat

The main steps of the code are the following: Iterate through the simulations, for each simulation: move the protein and its water molecules to positive coordinates, wrap it to avoid dispersion of water and align all frames in the simulation to the first frame of the first simulated molecule, count the frecuency of oxygen atoms in the voxels. As all simulations are aligned to the first one, we can attribute water distribution to the interaction of the protein and the water. 

Please, change the name 'simu_mainfolder' to the name of the folder where the simulations are stored. Also, change 'simu_mainfolder/*/structure.pdb' to the corresponding path to the protein .pdb file.

As you can see, a value of 54.0 was selected to move the protein. This value was obtained after running all the simulations trying to obtain the farthest point in one of the three dimensions, which turn out to be over -800. That will create a huge and innecesary matrix which can cause a "MemoryError". To avoid this, another value that was popular among the farthest was selected. In the "dense" function, you can see a condition that does not allow negative coordinates to enter values in the matrix. Less than ten molecules are excluded with this method.

Once the molecule is moved to [54.0,54.0,54.0] all coordinates of the molecule object (except by those few extreme values) have positive values, which eases the indexing of the matrix.

In [3]:
count2=0
sims=simlist(glob('simu_mainfolder/*/'),glob('simu_mainfolder/*/structure.pdb')) #Load structures and their corresponding trajectories.
tmax=54.0 #Frecuent maximun distance of atoms in one dimension from origin in all simulations. Obtained by empirical analysis with this dataset.
#main loop
for x in sims:
    print(count2,'/9')
    mol=Molecule(x)
    framesn=mol.numFrames
    atomsn=mol.numAtoms
    mol.wrap('protein') #wrap to avoid dispersion of water, in a biological system, water is not allow to flow away from proteins.
    mol.moveBy([tmax,tmax,tmax]) #now everything is in positive coordinates
    if count2>0:
        mol.align('protein',refmol) 
        #refmol is the first simulation in the data set.
        #Align all the simulations with the first allows water molecules to have the same spatial reference.
        mymatrix=dense(mol,mymatrix)
    else:
        mol.align('protein') 
        mymatrix=dense(mol)
    #mol.view() You can uncomment this line to view all simulations superposed.
    if count2==0:
        refmol=mol
    count2+=1

Creating simlist: 100% (11/11) [###################################] eta 00:01 -
0 /9


/home/alejandro/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


Now, we have to transform probabilities into energy terms and draw the cube. The line "mol.view()" will open VMD. Once it is open, load the waterpot.cube file to superpose it with the protein. If you open externally the protein, it will be placed at [0,0,0] and it will not superpose with the .cube file.

In the image, one can see the superposition between the protein and the water potential revealing a strong interaction between the two molecules. At the medium-right area of the sphere there is a part of the protein which is not "covered" by the water potential surface, this is due to the fact that we are observing one frame which is not very frequent.

In [4]:
kB=0.001987191
T=298
densitymat_norm=mymatrix/(framesn*10*atomsn) #data normalisation
densitymat_norm=densitymat_norm+(10**(-40)) #pseudocounts to avoid -inf results.
densitymat_norm=-kB*np.log(densitymat_norm)*T
writeVoxels(densitymat_norm,'waterpot.cube',np.array([0,0,0]),densitymat_norm.shape,np.array([1,1,1]))
mol.view()

<img src="Resultp2.png">